# Reconciliation #05: Loss Distribution Calibration

## Overview

This notebook verifies that the `ManufacturingLossGenerator` produces losses
whose empirical frequency and severity match the configured theoretical
parameters. Each of the three loss types (attritional, large, catastrophic)
is checked for statistical consistency using Kolmogorov-Smirnov tests,
moment comparisons, and CDF overlays.

**What is tested:**
- Attritional severity follows Lognormal(mean=25000, cv=1.5)
- Large severity follows Lognormal(mean=2000000, cv=2.0)
- Catastrophic severity follows Pareto(alpha=2.5, xm=1000000)
- Each loss type has frequency consistent with its Poisson rate
- Aggregate annual loss mean equals the sum of component means

**Prerequisites:** `ergodic_insurance` package installed

**Estimated runtime:** < 30 seconds

**Audience:** Developers, actuaries validating model calibration

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/reconciliation'

    if os.path.exists(os.path.join(NOTEBOOK_DIR, '_reconciliation_helpers.py')):
        print(f"Found helper module in {NOTEBOOK_DIR}")
        os.chdir(NOTEBOOK_DIR)
        if NOTEBOOK_DIR not in sys.path:
            sys.path.append(NOTEBOOK_DIR)
    else:
        print(f"WARNING: _reconciliation_helpers.py not found in {NOTEBOOK_DIR}")
        print("Please update NOTEBOOK_DIR in this cell to the correct folder path.")

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats as sp_stats

from ergodic_insurance.loss_distributions import (
    ManufacturingLossGenerator, LossEvent, LognormalLoss, ParetoLoss
)

notebook_header(
    5,
    "Loss Distribution Calibration",
    "Verify that generated losses match configured frequency and severity parameters."
)

## Configure Loss Generator

We use the default `ManufacturingLossGenerator` parameters at the reference
revenue of $10M so that frequency scaling factors are exactly 1.0. This
lets us compare directly against the base distribution parameters.

| Component | Frequency | Severity Distribution | Key Parameters |
|---|---|---|---|
| Attritional | 5.0/yr | Lognormal | mean=\$25,000, cv=1.5 |
| Large | 0.3/yr | Lognormal | mean=\$2,000,000, cv=2.0 |
| Catastrophic | 0.03/yr | Pareto | alpha=2.5, xm=\$1,000,000 |

In [ ]:
SEED = 42  # Fixed seed for reproducibility
REVENUE = 10_000_000  # Reference revenue -> scaling factor = 1.0
N_YEARS = 2_000  # Simulate many years to accumulate ~10K+ attritional events

# Default parameters (from class definitions)
ATTRITIONAL_FREQ = 5.0
ATTRITIONAL_MEAN = 25_000
ATTRITIONAL_CV = 1.5

LARGE_FREQ = 0.3
LARGE_MEAN = 2_000_000
LARGE_CV = 2.0

CAT_FREQ = 0.03
CAT_ALPHA = 2.5
CAT_XM = 1_000_000

# Theoretical Pareto mean: alpha * xm / (alpha - 1)
CAT_THEORETICAL_MEAN = CAT_ALPHA * CAT_XM / (CAT_ALPHA - 1)

# Theoretical lognormal std: mean * sqrt(exp(sigma^2) - 1) where sigma^2 = log(1 + cv^2)
def lognormal_std(mean, cv):
    """Theoretical standard deviation of a Lognormal(mean, cv) distribution."""
    return mean * cv  # by definition: cv = std / mean

# Pareto variance: xm^2 * alpha / ((alpha-1)^2 * (alpha-2)) for alpha > 2
CAT_THEORETICAL_VAR = (CAT_XM**2 * CAT_ALPHA) / ((CAT_ALPHA - 1)**2 * (CAT_ALPHA - 2))
CAT_THEORETICAL_STD = np.sqrt(CAT_THEORETICAL_VAR)

print(f"Simulation: {N_YEARS:,} years at ${REVENUE:,.0f} revenue")
print(f"Expected attritional events: ~{ATTRITIONAL_FREQ * N_YEARS:,.0f}")
print(f"Expected large events: ~{LARGE_FREQ * N_YEARS:,.0f}")
print(f"Expected catastrophic events: ~{CAT_FREQ * N_YEARS:,.0f}")
print(f"\nTheoretical means:")
print(f"  Attritional severity: ${ATTRITIONAL_MEAN:,.0f}")
print(f"  Large severity:       ${LARGE_MEAN:,.0f}")
print(f"  Catastrophic severity: ${CAT_THEORETICAL_MEAN:,.0f}")

## Generate Sample

Generate losses over the full simulation horizon in a single call, then
separate by loss type for analysis.

In [ ]:
with timed_cell("Generate losses"):
    generator = ManufacturingLossGenerator(seed=SEED)
    all_losses, sim_stats = generator.generate_losses(
        duration=N_YEARS, revenue=REVENUE, include_catastrophic=True
    )

# Separate by type
attritional_losses = [e for e in all_losses if e.loss_type == "attritional"]
large_losses = [e for e in all_losses if e.loss_type == "large"]
cat_losses = [e for e in all_losses if e.loss_type == "catastrophic"]

attritional_amounts = np.array([e.amount for e in attritional_losses])
large_amounts = np.array([e.amount for e in large_losses])
cat_amounts = np.array([e.amount for e in cat_losses])

print(f"Total events generated: {len(all_losses):,}")
print(f"  Attritional: {len(attritional_losses):,}")
print(f"  Large:       {len(large_losses):,}")
print(f"  Catastrophic: {len(cat_losses):,}")

---
## Attritional Losses

Attritional losses use a **Lognormal** severity distribution with
mean=$25,000 and CV=1.5. We compare the empirical moments against
theoretical values and run a KS test.

In [ ]:
section_header("Attritional Losses")
chk_att = ReconciliationChecker("Attritional Severity")

# --- Empirical statistics ---
emp_mean_att = np.mean(attritional_amounts)
emp_std_att = np.std(attritional_amounts, ddof=1)
n_att = len(attritional_amounts)

theo_mean_att = ATTRITIONAL_MEAN
theo_std_att = lognormal_std(ATTRITIONAL_MEAN, ATTRITIONAL_CV)

# Allow 10% relative tolerance for mean (generous for large samples)
mean_rel_err_att = abs(emp_mean_att - theo_mean_att) / theo_mean_att
chk_att.check(
    mean_rel_err_att < 0.10,
    "Attritional mean within 10% of theoretical",
    f"Empirical={fmt_dollar(emp_mean_att)}, Theoretical={fmt_dollar(theo_mean_att)}, RelErr={mean_rel_err_att:.4f}"
)

# Std within 15% (higher tolerance due to lognormal skew)
std_rel_err_att = abs(emp_std_att - theo_std_att) / theo_std_att
chk_att.check(
    std_rel_err_att < 0.15,
    "Attritional std within 15% of theoretical",
    f"Empirical={fmt_dollar(emp_std_att)}, Theoretical={fmt_dollar(theo_std_att)}, RelErr={std_rel_err_att:.4f}"
)

# --- KS test against theoretical lognormal ---
# Compute lognormal mu/sigma from mean and cv
sigma_att = np.sqrt(np.log(1 + ATTRITIONAL_CV**2))
mu_att = np.log(ATTRITIONAL_MEAN) - sigma_att**2 / 2

ks_stat_att, ks_p_att = sp_stats.kstest(
    attritional_amounts,
    lambda x: sp_stats.lognorm.cdf(x, s=sigma_att, scale=np.exp(mu_att))
)
chk_att.check(
    ks_p_att > 0.01,
    "KS test: attritional ~ Lognormal (p > 0.01)",
    f"KS stat={ks_stat_att:.4f}, p-value={ks_p_att:.4f}"
)

# --- Summary table ---
att_summary = pd.DataFrame({
    "Parameter": ["Mean", "Std Dev", "Count", "KS p-value"],
    "Expected": [f"${theo_mean_att:,.0f}", f"${theo_std_att:,.0f}",
                 f"~{ATTRITIONAL_FREQ * N_YEARS:,.0f}", "> 0.01"],
    "Observed": [f"${emp_mean_att:,.0f}", f"${emp_std_att:,.0f}",
                 f"{n_att:,}", f"{ks_p_att:.4f}"],
    "Status": [
        "PASS" if mean_rel_err_att < 0.10 else "FAIL",
        "PASS" if std_rel_err_att < 0.15 else "FAIL",
        "PASS" if abs(n_att - ATTRITIONAL_FREQ * N_YEARS) / (ATTRITIONAL_FREQ * N_YEARS) < 0.10 else "FAIL",
        "PASS" if ks_p_att > 0.01 else "FAIL",
    ]
})
display_df(att_summary, "Attritional Loss Calibration")

# --- CDF plot ---
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sorted_att = np.sort(attritional_amounts)
ecdf_att = np.arange(1, len(sorted_att) + 1) / len(sorted_att)
ax.plot(sorted_att, ecdf_att, label="Empirical CDF", linewidth=1.5)

x_theo = np.linspace(sorted_att.min(), np.percentile(sorted_att, 99.5), 500)
theo_cdf_att = sp_stats.lognorm.cdf(x_theo, s=sigma_att, scale=np.exp(mu_att))
ax.plot(x_theo, theo_cdf_att, '--', label="Theoretical Lognormal CDF", linewidth=1.5)

ax.set_xlabel("Loss Amount ($)")
ax.set_ylabel("Cumulative Probability")
ax.set_title("Attritional Losses: Empirical vs Theoretical CDF")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

chk_att.display_results()

---
## Large Losses

Large losses use a **Lognormal** severity distribution with
mean=$2,000,000 and CV=2.0.

In [ ]:
section_header("Large Losses")
chk_large = ReconciliationChecker("Large Loss Severity")

# --- Empirical statistics ---
emp_mean_lg = np.mean(large_amounts)
emp_std_lg = np.std(large_amounts, ddof=1)
n_lg = len(large_amounts)

theo_mean_lg = LARGE_MEAN
theo_std_lg = lognormal_std(LARGE_MEAN, LARGE_CV)

# Large losses have high CV so allow 20% tolerance for mean
mean_rel_err_lg = abs(emp_mean_lg - theo_mean_lg) / theo_mean_lg
chk_large.check(
    mean_rel_err_lg < 0.20,
    "Large loss mean within 20% of theoretical",
    f"Empirical={fmt_dollar(emp_mean_lg)}, Theoretical={fmt_dollar(theo_mean_lg)}, RelErr={mean_rel_err_lg:.4f}"
)

# Std within 30% (very high CV makes std estimation noisy)
std_rel_err_lg = abs(emp_std_lg - theo_std_lg) / theo_std_lg
chk_large.check(
    std_rel_err_lg < 0.30,
    "Large loss std within 30% of theoretical",
    f"Empirical={fmt_dollar(emp_std_lg)}, Theoretical={fmt_dollar(theo_std_lg)}, RelErr={std_rel_err_lg:.4f}"
)

# --- KS test ---
sigma_lg = np.sqrt(np.log(1 + LARGE_CV**2))
mu_lg = np.log(LARGE_MEAN) - sigma_lg**2 / 2

ks_stat_lg, ks_p_lg = sp_stats.kstest(
    large_amounts,
    lambda x: sp_stats.lognorm.cdf(x, s=sigma_lg, scale=np.exp(mu_lg))
)
chk_large.check(
    ks_p_lg > 0.01,
    "KS test: large ~ Lognormal (p > 0.01)",
    f"KS stat={ks_stat_lg:.4f}, p-value={ks_p_lg:.4f}"
)

# --- Summary table ---
lg_summary = pd.DataFrame({
    "Parameter": ["Mean", "Std Dev", "Count", "KS p-value"],
    "Expected": [f"${theo_mean_lg:,.0f}", f"${theo_std_lg:,.0f}",
                 f"~{LARGE_FREQ * N_YEARS:,.0f}", "> 0.01"],
    "Observed": [f"${emp_mean_lg:,.0f}", f"${emp_std_lg:,.0f}",
                 f"{n_lg:,}", f"{ks_p_lg:.4f}"],
    "Status": [
        "PASS" if mean_rel_err_lg < 0.20 else "FAIL",
        "PASS" if std_rel_err_lg < 0.30 else "FAIL",
        "PASS" if abs(n_lg - LARGE_FREQ * N_YEARS) / (LARGE_FREQ * N_YEARS) < 0.15 else "FAIL",
        "PASS" if ks_p_lg > 0.01 else "FAIL",
    ]
})
display_df(lg_summary, "Large Loss Calibration")

# --- CDF plot ---
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
sorted_lg = np.sort(large_amounts)
ecdf_lg = np.arange(1, len(sorted_lg) + 1) / len(sorted_lg)
ax.plot(sorted_lg, ecdf_lg, label="Empirical CDF", linewidth=1.5)

x_theo_lg = np.linspace(sorted_lg.min(), np.percentile(sorted_lg, 99.5), 500)
theo_cdf_lg = sp_stats.lognorm.cdf(x_theo_lg, s=sigma_lg, scale=np.exp(mu_lg))
ax.plot(x_theo_lg, theo_cdf_lg, '--', label="Theoretical Lognormal CDF", linewidth=1.5)

ax.set_xlabel("Loss Amount ($)")
ax.set_ylabel("Cumulative Probability")
ax.set_title("Large Losses: Empirical vs Theoretical CDF")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

chk_large.display_results()

---
## Catastrophic Losses

Catastrophic losses use a **Pareto** severity distribution with
alpha=2.5 and xm=$1,000,000. The theoretical mean is
alpha * xm / (alpha - 1) = $1,666,667. All sampled values must
be >= xm.

In [ ]:
section_header("Catastrophic Losses")
chk_cat = ReconciliationChecker("Catastrophic Severity")

# --- Basic sanity: all values >= xm ---
chk_cat.check(
    len(cat_amounts) > 0,
    "At least one catastrophic event generated",
    f"Count={len(cat_amounts)}"
)

if len(cat_amounts) > 0:
    chk_cat.check(
        np.all(cat_amounts >= CAT_XM),
        f"All catastrophic losses >= xm (${CAT_XM:,.0f})",
        f"Min observed={fmt_dollar(np.min(cat_amounts))}"
    )

    # --- Empirical statistics ---
    emp_mean_cat = np.mean(cat_amounts)
    emp_std_cat = np.std(cat_amounts, ddof=1)
    n_cat = len(cat_amounts)

    theo_mean_cat = CAT_THEORETICAL_MEAN
    theo_std_cat = CAT_THEORETICAL_STD

    # Pareto is heavy-tailed, so allow wider tolerance (30% for mean)
    mean_rel_err_cat = abs(emp_mean_cat - theo_mean_cat) / theo_mean_cat
    chk_cat.check(
        mean_rel_err_cat < 0.30,
        "Catastrophic mean within 30% of theoretical",
        f"Empirical={fmt_dollar(emp_mean_cat)}, Theoretical={fmt_dollar(theo_mean_cat)}, RelErr={mean_rel_err_cat:.4f}"
    )

    # --- KS test against Pareto ---
    # Pareto CDF: F(x) = 1 - (xm/x)^alpha for x >= xm
    def pareto_cdf(x):
        x = np.atleast_1d(np.asarray(x, dtype=float))
        result = np.zeros_like(x)
        mask = x >= CAT_XM
        result[mask] = 1.0 - (CAT_XM / x[mask]) ** CAT_ALPHA
        return result if len(x) > 1 else float(result[0])

    ks_stat_cat, ks_p_cat = sp_stats.kstest(cat_amounts, pareto_cdf)
    chk_cat.check(
        ks_p_cat > 0.01,
        "KS test: catastrophic ~ Pareto (p > 0.01)",
        f"KS stat={ks_stat_cat:.4f}, p-value={ks_p_cat:.4f}"
    )

    # --- Summary table ---
    cat_summary = pd.DataFrame({
        "Parameter": ["Mean", "Std Dev", "Min >= xm", "Count", "KS p-value"],
        "Expected": [
            f"${theo_mean_cat:,.0f}", f"${theo_std_cat:,.0f}",
            "Yes", f"~{CAT_FREQ * N_YEARS:,.0f}", "> 0.01"
        ],
        "Observed": [
            f"${emp_mean_cat:,.0f}", f"${emp_std_cat:,.0f}",
            "Yes" if np.all(cat_amounts >= CAT_XM) else "No",
            f"{n_cat:,}", f"{ks_p_cat:.4f}"
        ],
        "Status": [
            "PASS" if mean_rel_err_cat < 0.30 else "FAIL",
            "--",
            "PASS" if np.all(cat_amounts >= CAT_XM) else "FAIL",
            "PASS" if abs(n_cat - CAT_FREQ * N_YEARS) / max(CAT_FREQ * N_YEARS, 1) < 0.40 else "FAIL",
            "PASS" if ks_p_cat > 0.01 else "FAIL",
        ]
    })
    display_df(cat_summary, "Catastrophic Loss Calibration")

    # --- CDF plot ---
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    sorted_cat = np.sort(cat_amounts)
    ecdf_cat = np.arange(1, len(sorted_cat) + 1) / len(sorted_cat)
    ax.plot(sorted_cat, ecdf_cat, label="Empirical CDF", linewidth=1.5)

    x_theo_cat = np.linspace(CAT_XM, np.percentile(sorted_cat, 99.5), 500)
    theo_cdf_cat = 1.0 - (CAT_XM / x_theo_cat) ** CAT_ALPHA
    ax.plot(x_theo_cat, theo_cdf_cat, '--', label="Theoretical Pareto CDF", linewidth=1.5)

    ax.set_xlabel("Loss Amount ($)")
    ax.set_ylabel("Cumulative Probability")
    ax.set_title("Catastrophic Losses: Empirical vs Theoretical CDF")
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No catastrophic events generated (expected ~60 over 2000 years).")
    print("This is extremely unlikely and indicates a problem.")

chk_cat.display_results()

---
## Frequency Validation

Each loss type is generated via a Poisson process. We verify that the
empirical per-year event counts match the expected Poisson rate by
running a chi-square goodness-of-fit test on the observed yearly
frequency distribution.

In [ ]:
section_header("Frequency Validation")
chk_freq = ReconciliationChecker("Poisson Frequency")

def count_per_year(losses, n_years):
    """Bin loss events into integer year buckets and count per year."""
    counts = np.zeros(n_years, dtype=int)
    for e in losses:
        yr = int(e.time)
        if 0 <= yr < n_years:
            counts[yr] += 1
    return counts

def poisson_gof_test(observed_counts, expected_lambda, label):
    """Chi-square goodness-of-fit test against Poisson(lambda).
    
    Groups counts into bins to ensure adequate expected frequencies (>= 5).
    Returns (chi2_stat, p_value, passed).
    """
    max_k = max(int(expected_lambda + 4 * np.sqrt(expected_lambda)), int(np.max(observed_counts)) + 1)
    max_k = min(max_k, 50)  # Cap for efficiency
    
    # Bin observed counts
    bins = list(range(max_k + 1)) + [max_k + 100]  # last bin is overflow
    observed_hist, _ = np.histogram(observed_counts, bins=bins)
    
    # Expected counts from Poisson
    n_total = len(observed_counts)
    expected_probs = [sp_stats.poisson.pmf(k, expected_lambda) for k in range(max_k)]
    expected_probs.append(1.0 - sp_stats.poisson.cdf(max_k - 1, expected_lambda))  # overflow
    expected_hist = np.array(expected_probs) * n_total
    
    # Merge bins with expected < 5
    merged_obs = []
    merged_exp = []
    acc_obs = 0
    acc_exp = 0.0
    for o, e in zip(observed_hist, expected_hist):
        acc_obs += o
        acc_exp += e
        if acc_exp >= 5.0:
            merged_obs.append(acc_obs)
            merged_exp.append(acc_exp)
            acc_obs = 0
            acc_exp = 0.0
    # Fold remainder into last bin
    if acc_obs > 0 or acc_exp > 0:
        if merged_obs:
            merged_obs[-1] += acc_obs
            merged_exp[-1] += acc_exp
        else:
            merged_obs.append(acc_obs)
            merged_exp.append(acc_exp)
    
    merged_obs = np.array(merged_obs, dtype=float)
    merged_exp = np.array(merged_exp, dtype=float)
    
    if len(merged_obs) < 2:
        return 0.0, 1.0, True  # Not enough bins to test
    
    chi2, p_value = sp_stats.chisquare(merged_obs, merged_exp)
    return chi2, p_value, p_value > 0.01

# --- Attritional frequency ---
att_counts = count_per_year(attritional_losses, N_YEARS)
att_obs_lambda = np.mean(att_counts)
chi2_att, p_att, pass_att = poisson_gof_test(att_counts, ATTRITIONAL_FREQ, "Attritional")

chk_freq.check(
    abs(att_obs_lambda - ATTRITIONAL_FREQ) / ATTRITIONAL_FREQ < 0.05,
    f"Attritional avg frequency ~ {ATTRITIONAL_FREQ:.1f}/yr",
    f"Observed={att_obs_lambda:.3f}/yr, Expected={ATTRITIONAL_FREQ:.1f}/yr"
)
chk_freq.check(
    pass_att,
    "Attritional frequency ~ Poisson (chi-square p > 0.01)",
    f"chi2={chi2_att:.2f}, p={p_att:.4f}"
)

# --- Large frequency ---
lg_counts = count_per_year(large_losses, N_YEARS)
lg_obs_lambda = np.mean(lg_counts)
chi2_lg, p_lg, pass_lg = poisson_gof_test(lg_counts, LARGE_FREQ, "Large")

chk_freq.check(
    abs(lg_obs_lambda - LARGE_FREQ) / LARGE_FREQ < 0.10,
    f"Large avg frequency ~ {LARGE_FREQ:.1f}/yr",
    f"Observed={lg_obs_lambda:.3f}/yr, Expected={LARGE_FREQ:.1f}/yr"
)
chk_freq.check(
    pass_lg,
    "Large frequency ~ Poisson (chi-square p > 0.01)",
    f"chi2={chi2_lg:.2f}, p={p_lg:.4f}"
)

# --- Catastrophic frequency ---
cat_counts = count_per_year(cat_losses, N_YEARS)
cat_obs_lambda = np.mean(cat_counts)
# For very low-frequency events, use a simple mean comparison
chk_freq.check(
    abs(cat_obs_lambda - CAT_FREQ) / CAT_FREQ < 0.40,
    f"Catastrophic avg frequency ~ {CAT_FREQ:.2f}/yr",
    f"Observed={cat_obs_lambda:.4f}/yr, Expected={CAT_FREQ:.2f}/yr (40% tol for rare events)"
)

# Display frequency summary
freq_summary = pd.DataFrame({
    "Loss Type": ["Attritional", "Large", "Catastrophic"],
    "Expected Freq (/yr)": [ATTRITIONAL_FREQ, LARGE_FREQ, CAT_FREQ],
    "Observed Freq (/yr)": [att_obs_lambda, lg_obs_lambda, cat_obs_lambda],
    "Total Events": [len(attritional_losses), len(large_losses), len(cat_losses)],
    "Chi-sq p-value": [f"{p_att:.4f}", f"{p_lg:.4f}", "N/A (rare)"],
})
display_df(freq_summary, "Frequency Calibration Summary")

chk_freq.display_results()

---
## Aggregate Check

The combined annual expected loss should approximately equal the sum of
component expected losses:

$$E[L_{total}] = \lambda_{att} \cdot E[S_{att}] + \lambda_{lg} \cdot E[S_{lg}] + \lambda_{cat} \cdot E[S_{cat}]$$

In [ ]:
section_header("Aggregate Check")
chk_agg = ReconciliationChecker("Annual Aggregate")

# Theoretical annual expected loss per component
theo_annual_att = ATTRITIONAL_FREQ * ATTRITIONAL_MEAN
theo_annual_lg = LARGE_FREQ * LARGE_MEAN
theo_annual_cat = CAT_FREQ * CAT_THEORETICAL_MEAN
theo_annual_total = theo_annual_att + theo_annual_lg + theo_annual_cat

# Empirical annual loss
emp_total = sim_stats["total_amount"]
emp_annual = emp_total / N_YEARS

# Component annual losses
emp_annual_att = sim_stats["attritional_amount"] / N_YEARS
emp_annual_lg = sim_stats["large_amount"] / N_YEARS
emp_annual_cat = sim_stats["catastrophic_amount"] / N_YEARS

# Check aggregate mean within 15%
agg_rel_err = abs(emp_annual - theo_annual_total) / theo_annual_total
chk_agg.check(
    agg_rel_err < 0.15,
    "Aggregate annual loss within 15% of theoretical",
    f"Empirical={fmt_dollar(emp_annual)}/yr, Theoretical={fmt_dollar(theo_annual_total)}/yr, RelErr={agg_rel_err:.4f}"
)

# Check attritional component
att_agg_err = abs(emp_annual_att - theo_annual_att) / theo_annual_att
chk_agg.check(
    att_agg_err < 0.10,
    "Attritional annual loss within 10% of theoretical",
    f"Empirical={fmt_dollar(emp_annual_att)}/yr, Theoretical={fmt_dollar(theo_annual_att)}/yr"
)

# Check large component
lg_agg_err = abs(emp_annual_lg - theo_annual_lg) / theo_annual_lg
chk_agg.check(
    lg_agg_err < 0.20,
    "Large annual loss within 20% of theoretical",
    f"Empirical={fmt_dollar(emp_annual_lg)}/yr, Theoretical={fmt_dollar(theo_annual_lg)}/yr"
)

# Check that total == sum of components (exact, from simulation stats)
component_sum = sim_stats["attritional_amount"] + sim_stats["large_amount"] + sim_stats["catastrophic_amount"]
chk_agg.check(
    abs(emp_total - component_sum) < 0.01,
    "Total loss == sum of component losses (exact)",
    f"Total={fmt_dollar(emp_total)}, Sum={fmt_dollar(component_sum)}"
)

# Display aggregate summary
agg_summary = pd.DataFrame({
    "Component": ["Attritional", "Large", "Catastrophic", "TOTAL"],
    "Theoretical Annual": [
        fmt_dollar(theo_annual_att), fmt_dollar(theo_annual_lg),
        fmt_dollar(theo_annual_cat), fmt_dollar(theo_annual_total)
    ],
    "Empirical Annual": [
        fmt_dollar(emp_annual_att), fmt_dollar(emp_annual_lg),
        fmt_dollar(emp_annual_cat), fmt_dollar(emp_annual)
    ],
    "Relative Error": [
        f"{att_agg_err:.2%}", f"{lg_agg_err:.2%}",
        f"{abs(emp_annual_cat - theo_annual_cat) / max(theo_annual_cat, 1):.2%}",
        f"{agg_rel_err:.2%}"
    ],
})
display_df(agg_summary, "Annual Aggregate Loss Summary")

chk_agg.display_results()

---
## Final Summary

In [ ]:
final_summary(chk_att, chk_large, chk_cat, chk_freq, chk_agg)